Comparing MERRA2 geopotential product and calculated geopotential (from j Laughner) from GEOS products (geoschem) 

20160602 0000 time 


Data File Link for merra: https://daac.gsfc.nasa.gov/datasets/M2I3NVASM_5.12.4/summary?keywords=geopotential

Document containing summary of H variable and pressure table: 
https://gmao.gsfc.nasa.gov/pubs/docs/Bosilovich785.pdf


In [1]:
import xarray as xr 
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import bokeh 
import bokeh.io
import bokeh.plotting
from bokeh.plotting import figure
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
geos_calc = xr.open_dataset('../../data/GEOS/test_geopotentials.nc')

In [3]:
merra = xr.open_dataset('../../data/MERRA2/MERRA2_400.inst3_3d_asm_Nv.20160602.nc4')

In [4]:
merra

<xarray.Dataset>
Dimensions:  (lon: 576, lat: 361, lev: 72, time: 8)
Coordinates:
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 68.0 69.0 70.0 71.0 72.0
  * time     (time) datetime64[ns] 2016-06-02 ... 2016-06-02T21:00:00
Data variables: (12/17)
    CLOUD    (time, lev, lat, lon) float32 ...
    DELP     (time, lev, lat, lon) float32 ...
    EPV      (time, lev, lat, lon) float32 ...
    H        (time, lev, lat, lon) float32 ...
    O3       (time, lev, lat, lon) float32 ...
    OMEGA    (time, lev, lat, lon) float32 ...
    ...       ...
    QV       (time, lev, lat, lon) float32 ...
    RH       (time, lev, lat, lon) float32 ...
    SLP      (time, lat, lon) float32 ...
    T        (time, lev, lat, lon) float32 ...
    U        (time, lev, lat, lon) float32 ...
    V        (time, lev, lat, lon) float32 ...
Attributes: (12/30)
    History:                           Original file generated: Sat Jun 18 14...
    Comment:                           GMAO filename: d5124_m2_jan10.inst3_3d...
    Filename:                          MERRA2_400.inst3_3d_asm_Nv.20160602.nc4
    Conventions:                       CF-1
    Institution:                       NASA Global Modeling and Assimilation ...
    References:                        http://gmao.gsfc.nasa.gov
    ...                                ...
    Contact:                           http://gmao.gsfc.nasa.gov
    identifier_product_doi:            10.5067/WWQSXQ8IVFW8
    RangeBeginningDate:                2016-06-02
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   2016-06-02
    RangeEndingTime:                   21:00:00.000000

In [5]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]


p = bokeh.plotting.figure(height=fh, width=fw, title='')

for i in range(len(geos_calc.gph_from_pressure)):
    p.circle(geos_calc.level.values,
         geos_calc.gph_from_boxheight[i].values, size=5, 
         color=colors[2],legend_label='calc gph from boxheight')
    p.circle(geos_calc.level.values,
         geos_calc.gph_from_pressure[i].values, size=5, 
         color=colors[1],legend_label='calc gph from pressure')
    
p.xaxis.axis_label = "level"
p.yaxis.axis_label = "gph"
p.legend.location = "top_left"
bokeh.io.show(p)


In [6]:
merra.H.values.shape

(8, 72, 361, 576)

In [7]:
merra.H.values[0].shape

(72, 361, 576)

Interpolating merra2 to josh grid just to make it easier

In [8]:
merrainterpped = merra.interp(lon=geos_calc.lon.values, lat=geos_calc.lat.values, method='nearest')
merrainterpped

<xarray.Dataset>
Dimensions:  (lev: 72, time: 8, lat: 16, lon: 16)
Coordinates:
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 68.0 69.0 70.0 71.0 72.0
  * time     (time) datetime64[ns] 2016-06-02 ... 2016-06-02T21:00:00
  * lon      (lon) float64 -100.0 -95.0 -100.0 -95.0 10.0 ... -5.0 0.0 -5.0 0.0
  * lat      (lat) float64 34.0 34.0 38.0 38.0 46.0 ... 89.0 89.0 -89.0 -89.0
Data variables: (12/17)
    CLOUD    (time, lev, lat, lon) float32 0.0 0.0 0.0 ... 0.2681 0.256 0.2681
    DELP     (time, lev, lat, lon) float32 1.0 1.0 1.0 ... 1.029e+03 1.028e+03
    EPV      (time, lev, lat, lon) float32 0.6804 0.6106 ... -2.199e-05
    H        (time, lev, lat, lon) float32 7.669e+04 7.663e+04 ... 2.749e+03
    O3       (time, lev, lat, lon) float32 8.682e-08 8.603e-08 ... 7.344e-08
    OMEGA    (time, lev, lat, lon) float32 6.111e-06 -1.626e-06 ... 0.04204
    ...       ...
    QV       (time, lev, lat, lon) float32 2.254e-06 2.365e-06 ... 1.694e-05
    RH       (time, lev, lat, lon) float32 3.359e-05 4.324e-05 ... 0.9706 0.9723
    SLP      (time, lat, lon) float32 1.012e+05 1.013e+05 ... 9.834e+04
    T        (time, lev, lat, lon) float32 196.7 195.5 196.7 ... 217.3 217.4
    U        (time, lev, lat, lon) float32 -25.3 -32.39 -25.3 ... -6.972 -6.713
    V        (time, lev, lat, lon) float32 -2.163 -0.3376 ... -4.229 -4.91
Attributes: (12/30)
    History:                           Original file generated: Sat Jun 18 14...
    Comment:                           GMAO filename: d5124_m2_jan10.inst3_3d...
    Filename:                          MERRA2_400.inst3_3d_asm_Nv.20160602.nc4
    Conventions:                       CF-1
    Institution:                       NASA Global Modeling and Assimilation ...
    References:                        http://gmao.gsfc.nasa.gov
    ...                                ...
    Contact:                           http://gmao.gsfc.nasa.gov
    identifier_product_doi:            10.5067/WWQSXQ8IVFW8
    RangeBeginningDate:                2016-06-02
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   2016-06-02
    RangeEndingTime:                   21:00:00.000000

In [9]:
merrainterpped.lev.values

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
       40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52.,
       53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65.,
       66., 67., 68., 69., 70., 71., 72.])

In [10]:
geos_calc.level.values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71])

Here is the table that shows what pressure the level corresponds to in merra 2: https://gmao.gsfc.nasa.gov/pubs/docs/Bosilovich785.pdf 

In [11]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]
levarray = list(range(0,72)) # matching Josh's

p = bokeh.plotting.figure(height=fh, width=fw, title='')

for i in range(len(geos_calc.gph_from_pressure)):    
    p.circle(geos_calc.level.values,
         geos_calc.gph_from_boxheight[i].values, size=5, 
         color=colors[2],legend_label='calc gph from boxheight')
    p.circle(geos_calc.level.values,
         geos_calc.gph_from_pressure[i].values, size=5, 
         color=colors[1],legend_label='calc gph from pressure')


    # array is time,lev,lat,lon in that order. Getting time 0
    # the level is opposite (low pressure at level 1) compared to Josh output
    # so plotting against a level array I made but backwards
    p.circle(levarray[::-1],merrainterpped.H.values[0,:,i,i]/1000,
             size=5,color=colors[4],legend_label='merra2')

p.xaxis.axis_label = "level"
p.yaxis.axis_label = "gph"
p.legend.location = "top_left"
bokeh.io.show(p)



In [12]:
r1ar = np.zeros((16,72))
r2ar = np.zeros((16,72))

for i in range(len(geos_calc.gph_from_pressure)):  
    r1 = geos_calc.gph_from_pressure[i].values - (merrainterpped.H.values[0,:,i,i]/1000)[::-1]
    r2 = geos_calc.gph_from_boxheight[i].values - (merrainterpped.H.values[0,:,i,i]/1000)[::-1]
    
    r1ar[i,:] = r1
    r2ar[i,:] = r2

In [13]:
fh = 500
fw = 400
colors = bokeh.palettes.d3['Category20'][20]
levarray = list(range(0,72))

p = bokeh.plotting.figure(height=fh, width=fw, title='')

for i in range(len(geos_calc.gph_from_pressure)):  
    r1 = geos_calc.gph_from_pressure[i].values - (merrainterpped.H.values[0,:,i,i]/1000)[::-1]
    r2 = geos_calc.gph_from_boxheight[i].values - (merrainterpped.H.values[0,:,i,i]/1000)[::-1]
    
    p.circle(geos_calc.level.values, r1, size=5, color=colors[3], legend_label='gph p - merra2')
    p.circle(geos_calc.level.values, r2, size=5, color=colors[1], legend_label='gph b - merra2')

    
p.circle(geos_calc.level.values,np.mean(r1ar,axis=0), size=5, color=colors[2], legend_label='mean diff p')  
p.circle(geos_calc.level.values, np.mean(r2ar, axis=0), size=5, color=colors[0], legend_label='mean diff b')
    
p.xaxis.axis_label = "level"
p.yaxis.axis_label = "gph"
p.legend.location = "top_left"
bokeh.io.show(p)



Level 32 corresponds to 100hPa https://gmao.gsfc.nasa.gov/pubs/docs/Bosilovich785.pdf 

So for levels from 0-40 (where it will matter more for e and p) perhaps pressure calculation is more accurate. 